In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models, optimizers 
from tensorflow.keras.layers import Input, Add, subtract, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from scipy.io import loadmat
import pandas as pd


In [174]:
def FFDnet(input_shape = (5, 51, 5), depth=10, feature_map=32):
    X_input= Input(shape=input_shape)
    X_shortcut=X_input[:, :, :,0:4]
    
    X = Conv2D(feature_map, (3, 3), padding='same', input_shape=(5,51,5), activation='relu')(X_input)
    for i in range(depth-2):
        X = Conv2D(feature_map, (3, 3), padding='same')(X)
        X = BatchNormalization()(X)
        X = Activation('relu')(X)
    
    X = Conv2D(4, (3, 3), padding='same')(X)
    X = subtract([X_shortcut, X])
    
    model = Model(inputs = X_input, outputs = X, name='FFDnet')

    return model

    

In [175]:
model = FFDnet(input_shape = (5, 51, 5), depth=10, feature_map=32)

In [176]:
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [383]:
model.load_weights('model_depth10_FM8.h5')

In [177]:
#!pip install mat73
import mat73
data_dict = mat73.loadmat('../nDatasetLIS_M10_K50_LIS.mat')

In [178]:
#import pickle

#with open('Dataset_LIS_M10_K50.pickle', 'wb') as handle:
    #pickle.dump(data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [179]:
# from google.colab import drive
# drive.mount('/content/drive')

In [180]:
# import pickle
# with open('/content/drive/My Drive/Dataset_LIS_M10_K50.pickle', 'rb') as handle:
#     data_dict = pickle.load(handle)

In [181]:
data_dict.keys()

dict_keys(['Dataset_M10_K50_LIS'])

In [182]:
Data_H=data_dict['Dataset_M10_K50_LIS']

In [183]:
Data_H.keys()

dict_keys(['H_ls', 'H_true', 'K', 'M', 'Tp', 'noise_map'])

In [184]:
Data_H_ls=Data_H['H_ls']

In [185]:
Data_H_true=Data_H['H_true']

In [186]:
Data_noise_mp=Data_H['noise_map']

In [187]:
np.shape(Data_H_ls)

(60000, 51, 10, 2)

In [188]:
Data_H_ls=np.transpose(Data_H_ls,(0,2,1,3))

In [189]:
np.shape(Data_H_ls)

(60000, 10, 51, 2)

In [190]:
#Data_H_ls=np.reshape(Data_H_ls,(30000,2,10,51))

In [191]:
np.shape(Data_H_ls)

(60000, 10, 51, 2)

In [192]:
#Data_H_true=np.transpose(Data_H_true)
Data_H_true=np.transpose(Data_H_true,(0,2,1,3))

In [193]:
np.shape(Data_H_true)

(60000, 10, 51, 2)

In [194]:
Data_noise_mp=np.transpose(Data_noise_mp)
np.shape(Data_noise_mp)

(1, 60000)

In [195]:
Data_noise_mp=np.transpose(Data_noise_mp)
np.shape(Data_noise_mp)

(60000, 1)

In [196]:
idx=np.random.permutation(np.shape(Data_H_true)[0])

In [197]:
shuffled_H_ls=Data_H_ls[idx]
np.shape(shuffled_H_ls)

(60000, 10, 51, 2)

In [198]:
shuffled_H_true=Data_H_true[idx]
np.shape(shuffled_H_true)

(60000, 10, 51, 2)

In [199]:
shuffled_noise_map=Data_noise_mp[idx]
np.shape(shuffled_noise_map)

(60000, 1)

In [200]:
train_H_ls=shuffled_H_ls[0:54000,:,:,:]
np.shape(train_H_ls)

(54000, 10, 51, 2)

In [201]:
validation_H_ls=shuffled_H_ls[54000:58000,:,:,:]
np.shape(validation_H_ls)

(4000, 10, 51, 2)

In [202]:
test_H_ls=shuffled_H_ls[58000:60000,:,:,:]
np.shape(test_H_ls)

(2000, 10, 51, 2)

In [203]:
train_H_true=shuffled_H_true[0:54000,:,:,:]
validation_H_true=shuffled_H_true[54000:58000,:,:,:]
test_H_true=shuffled_H_true[58000:60000,:,:,:]

In [204]:
train_noise_map=shuffled_noise_map[0:54000]
validation_noise_map=shuffled_noise_map[54000:58000]
test_noise_map=shuffled_noise_map[58000:60000]

In [205]:
train_H_true=np.reshape(train_H_true,(train_H_true.shape[0],5,51,4)) #channel last
np.shape(train_H_true)

(54000, 5, 51, 4)

In [206]:
validation_H_true=np.reshape(validation_H_true,(validation_H_true.shape[0],5,51,4))
test_H_true=np.reshape(test_H_true,(test_H_true.shape[0],5,51,4))

In [207]:
train_H_ls=np.reshape(train_H_ls,(train_H_ls.shape[0],5,51,4))
validation_H_ls=np.reshape(validation_H_ls,(validation_H_ls.shape[0],5,51,4))
test_H_ls=np.reshape(test_H_ls,(test_H_ls.shape[0],5,51,4))

In [208]:
train_awgn_noise_map=np.zeros((train_noise_map.shape[0],5,51))
np.shape(train_awgn_noise_map)

(54000, 5, 51)

In [209]:
for i in range(train_awgn_noise_map.shape[0]):
    train_awgn_noise_map[i,:,:]=train_noise_map[i]*np.ones((5,51))

np.shape(train_awgn_noise_map)

(54000, 5, 51)

In [210]:
validation_awgn_noise_map=np.zeros((validation_noise_map.shape[0],5,51))

for i in range(validation_awgn_noise_map.shape[0]):
    validation_awgn_noise_map[i,:,:]=validation_noise_map[i]*np.ones((5,51))

np.shape(validation_awgn_noise_map)

(4000, 5, 51)

In [211]:
test_awgn_noise_map=np.zeros((test_noise_map.shape[0],5,51))

for i in range(test_awgn_noise_map.shape[0]):
    test_awgn_noise_map[i,:,:]=test_noise_map[i]*np.ones((5,51))

np.shape(test_awgn_noise_map)

(2000, 5, 51)

In [212]:
X_train=np.zeros((train_H_ls.shape[0],train_H_ls.shape[1],train_H_ls.shape[2],5))
np.shape(X_train)

(54000, 5, 51, 5)

In [213]:
X_train[:,:,:,0:4]=train_H_ls


In [214]:
X_train[:,:,:,4]=np.reshape(train_awgn_noise_map,(train_awgn_noise_map.shape[0],train_awgn_noise_map.shape[1],train_awgn_noise_map.shape[2]))

In [215]:
X_validation=np.zeros((validation_H_ls.shape[0],validation_H_ls.shape[1],validation_H_ls.shape[2],5))
X_validation[:,:,:,0:4]=validation_H_ls
X_validation[:,:,:,4]=np.reshape(validation_awgn_noise_map,(validation_awgn_noise_map.shape[0],validation_awgn_noise_map.shape[1],validation_awgn_noise_map.shape[2]))

In [216]:
X_test=np.zeros((test_H_ls.shape[0],test_H_ls.shape[1],test_H_ls.shape[2],5))
X_test[:,:,:,0:4]=test_H_ls
X_test[:,:,:,4]=np.reshape(test_awgn_noise_map,(test_awgn_noise_map.shape[0],test_awgn_noise_map.shape[1],test_awgn_noise_map.shape[2]))

In [217]:
Y_train=train_H_true
Y_validation=validation_H_true
Y_test=test_H_true

In [218]:
import sklearn.preprocessing

In [219]:

X_train1=np.interp(X_train, (X_train.min(), X_train.max()), (0, 100))
Y_train1=np.interp(Y_train, (Y_train.min(), Y_train.max()), (0, 100))
X_validation1=np.interp(X_validation, (X_validation.min(), X_validation.max()), (0, 100))
Y_validation1=np.interp(Y_validation, (Y_validation.min(), Y_validation.max()), (0, 100))
X_test1=np.interp(X_test, (X_test.min(), X_test.max()), (0, 100))
Y_test1=np.interp(X_test, (Y_test.min(), Y_test.max()), (0, 100))

In [220]:
EPOCHS=10
#training
batch_sz = 100

model.fit(X_train1,Y_train1, epochs=EPOCHS, verbose=1,validation_data=(X_validation1,Y_validation1),batch_size = batch_sz)
#save to disk
model_json = model.to_json()
with open('model_depth8_FM4.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model_depth8_FM4.h5') 

#test
scores = model.evaluate(X_test, Y_test, batch_size=80, verbose=1)
print('\nTest MSE: %.3f loss: %.3f' % (scores[0],scores[1])) 

Train on 54000 samples, validate on 4000 samples
Epoch 1/10
54000/54000 [==============================] - 71s 1ms/sample - loss: 0.1426 - mse: 0.1426 - val_loss: 0.4160 - val_mse: 0.4160
Epoch 2/10
54000/54000 [==============================] - 58s 1ms/sample - loss: 0.1392 - mse: 0.1392 - val_loss: 0.4181 - val_mse: 0.4181
Epoch 3/10
54000/54000 [==============================] - 55s 1ms/sample - loss: 0.1387 - mse: 0.1387 - val_loss: 0.4168 - val_mse: 0.4168
Epoch 4/10
54000/54000 [==============================] - 55s 1ms/sample - loss: 0.1378 - mse: 0.1378 - val_loss: 0.4140 - val_mse: 0.4140
Epoch 5/10
54000/54000 [==============================] - 57s 1ms/sample - loss: 0.1373 - mse: 0.1373 - val_loss: 0.4105 - val_mse: 0.4105
Epoch 6/10
54000/54000 [==============================] - 58s 1ms/sample - loss: 0.1371 - mse: 0.1371 - val_loss: 0.4202 - val_mse: 0.4202
Epoch 7/10
54000/54000 [==============================] - 59s 1ms/sample - loss: 0.1368 - mse: 0.1368 - val_loss: 5.1


Test MSE: 0.098 loss: 0.098


In [222]:
snr=np.arange(start=0, stop=16, step=1)
LIS_FFDNet_MSE_depth8_FM32=np.zeros(shape=np.shape(snr))

In [223]:
# !pip install mat73
import mat73
for i in range(np.size(snr)):
    data_SNR = mat73.loadmat('../testing_dataset/test_DatasetLIS_M10_K50_LIS_SNR_{}.mat'.format(snr[i]))
    nData_H=data_SNR['test_Dataset_M10_K50_LIS_SNR{}'.format(snr[i])]
    nH_ls=nData_H['H_ls']
    nH_ls=np.transpose(nH_ls,(0,2,1,3))
    nH_true=nData_H['H_true']
    nH_true=np.transpose(nH_true,(0,2,1,3))
    nX_test=nH_ls
    nY_test=nH_true
    nX_test1=np.interp(nX_test, (nX_test.min(), nX_test.max()), (0, 100))
    nY_test1=np.interp(nY_test, (nY_test.min(), nY_test.max()), (0, 100))
    scores = model.evaluate(nX_test1, nY_test1, batch_size=100, verbose=1)
    LIS_FFDNet_MSE_depth8_FM32[i]=scores[1]
  

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

...

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [224]:
LIS_FFDNet_MSE_depth8_FM32

array([1.17807078, 0.93070799, 0.67692631, 0.91091985, 0.41354287,
       0.27133277, 0.20593908, 0.51775688, 0.33223489, 0.31875712,
       0.24503578, 0.34583747, 0.28804827, 0.13411225, 0.1456224 ,
       0.18467672])

In [113]:
LIS_FFDNet_MSE_depth8_FM4

array([0.00980944, 0.00778499, 0.00619045, 0.00491163, 0.00390697,
       0.00310142, 0.00245877, 0.00195505, 0.00155455, 0.00123499,
       0.0009805 , 0.00077851, 0.00061927, 0.00049165, 0.00038977,
       0.00030991])

In [57]:
LIS_FFDNet_MSE_depth12_FM4

array([0.00990222, 0.00789191, 0.00632526, 0.00503516, 0.00403236,
       0.00323627, 0.00258157, 0.00209002, 0.00166168, 0.0013778 ,
       0.00112915, 0.0009159 , 0.00075969, 0.00063815, 0.00052343,
       0.00045389])

In [46]:
LIS_FFDNet_MSE_depth8_FM4

array([0.00982007, 0.00779599, 0.00620128, 0.00492335, 0.00391874,
       0.00311251, 0.00247029, 0.00196669, 0.00156582, 0.00124627,
       0.00099164, 0.00079025, 0.00063099, 0.0005029 , 0.00040117,
       0.00032145])

In [52]:
print(LIS_FFDNet_MSE_depth4_FM4)

[0.00985976 0.00783884 0.00624329 0.00496489 0.00396042 0.0031552
 0.00251327 0.00200901 0.00160961 0.00129048 0.00103693]


In [61]:
print(LIS_FFDNet_MSE_depth6_FM4)

[0.0098377  0.00781496 0.00621868 0.00494037 0.00393649 0.00313094
 0.00248823 0.00198479 0.00158521 0.00126468 0.00101034]


In [69]:
print(LIS_FFDNet_MSE_depth8_FM4)

[0.00981931 0.00779506 0.00620157 0.00492159 0.00391842 0.00311223
 0.00246959 0.00196626 0.00156553 0.00124637 0.00099179]


In [7]:
print(LIS_FFDNet_MSE_depth8_FM4)

[0.00991867 0.0078948  0.00630126 0.00502229 0.00401673 0.00321027
 0.00256599 0.002062   0.00166119 0.00134164 0.00109037 0.00088753
 0.00072753 0.00059942 0.0004962  0.00041595]


In [19]:
print(LIS_FFDNet_MSE_depth6_FM4)

[0.01137364 0.00938429 0.00776934 0.00646305 0.00544882 0.00463747
 0.00398607 0.00348431 0.00309468 0.00276724 0.00250043 0.00230277
 0.00213953 0.00201123 0.00190713 0.00182665]


In [26]:
print(LIS_FFDNet_MSE_depth6_FM4)

[0.0145965  0.01251443 0.01116982 0.00975678 0.00881853 0.00793083
 0.00730547 0.00673474 0.00648215 0.00608386 0.00577501 0.00568315
 0.00559893 0.00538034 0.005293   0.00514279]


In [36]:
print(LIS_FFDNet_MSE_depth6_FM4)

[0.0145965  0.01251443 0.01116982 0.00975678 0.00881853 0.00793083
 0.00730547 0.00673474 0.00648215 0.00608386 0.00577501 0.00568315
 0.00559893 0.00538034 0.005293   0.00514279]


In [33]:
LIS_FFDNet_MSE_depth6_FM4= np.array([0.0145965 ,0.01251443, 0.01116982, 0.00975678, 0.00881853, 0.00793083,
 0.00730547, 0.00673474, 0.00648215, 0.00608386, 0.00577501, 0.00568315,
 0.00559893, 0.00538034, 0.005293,   0.00514279])
print(LIS_FFDNet_MSE_depth6_FM4)

[0.0145965  0.01251443 0.01116982 0.00975678 0.00881853 0.00793083
 0.00730547 0.00673474 0.00648215 0.00608386 0.00577501 0.00568315
 0.00559893 0.00538034 0.005293   0.00514279]


In [44]:
print(LIS_FFDNet_MSE_depth8_FM4)

[0.00981931 0.00779506 0.00620157 0.00492159 0.00391842 0.00311223
 0.00246959 0.00196626 0.00156553 0.00124637 0.00099179 0.00078966
 0.00063099 0.00050285 0.00040116 0.00032143]


In [61]:
print(LIS_FFDNet_MSE_depth6_FM4)

[0.0098377  0.00781496 0.00621868 0.00494037 0.00393649 0.00313094
 0.00248823 0.00198479 0.00158521 0.00126468 0.00101034 0.00080846
 0.00065005 0.00052193 0.00042015 0.0003406 ]


In [59]:
print(LIS_FFDNet_MSE_depth8_FM4)

[0.00981931 0.00779506 0.00620157 0.00492159 0.00391842 0.00311223
 0.00246959 0.00196626 0.00156553 0.00124637 0.00099179 0.00078966
 0.00063099 0.00050285 0.00040116 0.00032143]


In [62]:
SNR=snr
import scipy.io
# [SNR, MSE_SNR_depth2_FM32]=pickle.load( open( "MSE_SNR_depth2_FM32.pkl", "rb" ) )
scipy.io.savemat('MSE_SNR_depth6_FM4v3.mat', mdict={'MSE_SNR_depth6_FM4': LIS_FFDNet_MSE_depth6_FM4,'SNR':SNR})

In [384]:
import time
import mat73
snrtest=15
data_SNR = mat73.loadmat('../testing_dataset/test_DatasetLIS_M10_K50_LIS_SNR_{}.mat'.format(snrtest))
Data_H=data_SNR['test_Dataset_M10_K50_LIS_SNR{}'.format(snrtest)]
H_ls=Data_H['H_ls']
H_ls=np.transpose(H_ls,(0,2,1,3))
H_true=Data_H['H_true']
H_true=np.transpose(H_true,(0,2,1,3))
X_test=H_ls
Y_test=H_true


In [385]:
start_time=time.time()
scores = model.evaluate(X_test, Y_test, batch_size=100, verbose=1)
print("--- %s milliseconds ---" % ((time.time() - start_time)*1000))
print(10*np.log10(scores[1]))

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

--- 400.90036392211914 milliseconds ---
-34.85388517379761
